In [1]:
import root_numpy
import numpy as np
from rootpy.io import root_open
from trainlib.FileCollection import FileCollection
from trainlib.utils import read_data
from trainlib.config import Config
from trainlib.SimpleModel import SimpleModel
import trainlib.utils as utils
import pandas as pd
import os
import trainlib.cuts

Welcome to JupyROOT 6.10/09


Using TensorFlow backend.


In [2]:
from trainlib.ModelCollection import ModelCollection

In [3]:
def no_cut(row):
    return True

In [11]:
inpath = {"/data_CMS/cms/wind/processed/ggH125/ZZ4lAnalysis.root": no_cut, "/data_CMS/cms/wind/processed/VBFH125/ZZ4lAnalysis.root": trainlib.cuts.mZZ_cut}

In [12]:
coll = FileCollection(inpath, 0.0, 1.0)

skimming /data_CMS/cms/wind/processed/ggH125/ZZ4lAnalysis.root
skimming /data_CMS/cms/wind/processed/VBFH125/ZZ4lAnalysis.root
collection set up: 2 files, 172803 entries in total, 172803 of which will be used


In [5]:
length = utils.get_size(inpath)

skimming /data_CMS/cms/wind/processed/ZZ4lAnalysis.root


In [6]:
length2 = coll.get_length()

In [18]:
indata = read_data(coll, start = 172750, stop = 172800, branches = Config.branches)

In [19]:
indata

,PFMET,nCleanedJetsPt30,nCleanedJetsPt30BTagged_bTagSF,nExtraLep,ZZMass,LHEAssociatedParticleId,GenAssocLep1Id,GenAssocLep2Id,D_VBF2j_ggH_ME,D_VBF1j_ggH_ME,D_WHh_ggH_ME,D_ZHh_ggH_ME,D_WHh_ZHh_ME,D_VBF2j_WHh_ME,D_VBF2j_ZHh_ME
0,32.060566,2,0,0,122.425636,"[1, -1, 21]",0,0,0.873086,0.222291,0.000228,3.591890e-05,0.864089,9.999668e-01,9.999948e-01
1,30.841005,2,0,0,123.186897,"[1, -1, 21]",0,0,0.791113,0.221478,0.000061,1.406474e-05,0.811960,9.999840e-01,9.999963e-01
2,25.694113,1,0,0,123.121223,"[2, -3, 3]",0,0,0.466257,0.567998,1.000000,1.000000e+00,0.032885,3.427478e-07,1.165458e-08
3,30.348368,0,0,0,124.375854,"[1, -1, 21]",0,0,0.466884,0.220792,1.000000,1.000000e+00,0.041512,3.782662e-07,1.638255e-08
4,27.868950,1,0,0,125.339844,"[2, 21, -2]",0,0,0.467815,0.710850,1.000000,1.000000e+00,0.061372,3.252219e-07,2.126446e-08
5,15.490951,2,0,0,123.144386,"[-1, 1, 21]",0,0,0.832383,0.221516,0.000739,1.805061e-04,0.803848,9.998510e-01,9.999636e-01
6,7.841118,4,0,0,124.742783,"[1, 21, -3]",0,0,0.154791,0.220746,0.995825,9.910548e-01,0.682814,7.672770e-04,1.650278e-03
7,5.943734,1,0,0,124.160866,"[2, 3, 21]",0,0,0.466732,0.005440,1.000000,1.000000e+00,0.039094,3.804933e-07,1.548015e-08
8,6.386643,2,0,0,124.770775,"[1, 4, -3]",0,0,0.680138,0.220745,0.000006,1.242815e-06,0.828373,9.999972e-01,9.999994e-01
9,53.078201,2,0,0,123.584351,"[4, 21, 1]",0,0,0.022013,0.221163,0.164813,1.163272e-01,0.599849,1.023826e-01,1.460164e-01


In [8]:
def trivial_preprocessor(frame):
    return frame

def sample_preprocessor(frame):
    return frame.loc[frame["nCleanedJetsPt30"] >= 2]

def complementary_preprocessor(frame):
    return frame.loc[frame["nCleanedJetsPt30"] < 2]

In [9]:
mcoll = ModelCollection("simplecoll")

In [10]:
mod = SimpleModel("simplemodel")
mod.build()
#mod.load("/home/llr/cms/wind/cmssw/CMSSW_9_4_2/src/ZZAnalysis/AnalysisStep/test/Python/logs/", "final.hdf5")

In [11]:
mcoll.add_model(trivial_preprocessor, mod)

In [12]:
mcoll.load_weights("/home/llr/cms/wind/cmssw/CMSSW_9_4_2/src/ZZAnalysis/AnalysisStep/test/Python/training_logs/simplecoll")

found the following models belonging to this collection:
simplemodel
now attempting to load model 'simplemodel' from file '/home/llr/cms/wind/cmssw/CMSSW_9_4_2/src/ZZAnalysis/AnalysisStep/test/Python/training_logs/simplecoll/simplemodel/final.hdf5


2018-03-20 16:43:04.593105: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.1 instructions, but these are available on your machine and could speed up CPU computations.
2018-03-20 16:43:04.593144: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use SSE4.2 instructions, but these are available on your machine and could speed up CPU computations.
2018-03-20 16:43:04.593160: W tensorflow/core/platform/cpu_feature_guard.cc:45] The TensorFlow library wasn't compiled to use AVX instructions, but these are available on your machine and could speed up CPU computations.


In [14]:
out_data = mcoll.predict(indata)

In [15]:
np.shape(out_data)

(110483,)

In [16]:
infile = root_open(inpath, mode = "a")
intree = infile.Get("ZZTree/candTree")

In [17]:
new_columns = np.array(out_data, dtype = [('classifier_output', 'f4')])

In [18]:
root_numpy.array2tree(new_columns, tree = intree)

<ROOT.TTree object ("candTree") at 0x7599b90>

In [19]:
infile.write()

81882

In [20]:
infile.close()

False

In [30]:
a_pre = np.array([1,2,3])
b_pre = np.array([4,5,6])

In [48]:
a = np.array(a_pre, dtype = [('test_a', 'f4')])

In [49]:
b = np.array(b_pre, dtype = [('test_b', 'f4')])

In [50]:
a = np.expand_dims(a, axis = 0)
b = np.expand_dims(b, axis = 0)

In [51]:
np.concatenate((a,b), axis = 0)

TypeError: invalid type promotion

In [53]:
np.array([a_pre, b_pre], dtype = [('test_a', 'f4'),('test_b', 'f4')])

array([[( 1.,  1.), ( 2.,  2.), ( 3.,  3.)],
       [( 4.,  4.), ( 5.,  5.), ( 6.,  6.)]], 
      dtype=[('test_a', '<f4'), ('test_b', '<f4')])

In [54]:
np.array([a_pre], dtype = [('test_a', 'f4')])

array([[( 1.,), ( 2.,), ( 3.,)]], 
      dtype=[('test_a', '<f4')])

In [55]:
def test(model):
    lalala = model()

In [56]:
class testclass:
    def __init__(self):
        print "initialized"

In [57]:
test(testclass)

initialized
